In [ ]:
# jupyter notebook tested on docker tensorflow/tensorflow:2.4.1-gpu-jupyter

import glob, shutil, os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.layers.experimental import preprocessing
from keras_preprocessing.image import ImageDataGenerator

from pandas import Series, DataFrame
from PIL import Image, ImageEnhance, ImageOps

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from tensorflow.keras.mixed_precision import experimental as mixed_precision
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = '1'

np.random.seed(0)
%matplotlib inline

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
### class list ###
annotated_group = ['AMY',
 'ANC',
 'ATN',
 'BNS',
 'CRE',
 'DMN',
 'END',
 'FAB',
 'FGS',
 'IGA',
 'LUD',
 'LUE',
 'LUX',
 'MCN',
 'MEN',
 'MGA',
 'MPG',
 'MSP',
 'PUR',
 'TIN',
 'XXX_Blank']

#　correspondence table　(Figure 7D)
# AMY = Amy
# ANC = ANCA
# ATN = ATN
# BNS = BNS
# CRE = Crescent
# DMN = DMN
# END = Endocap
# FAB = Fabry
# FGS = FSGS
# IGA = IgAN
# LUD = Lupus_IV
# LUE = Lupus_V
# LUX = Lupus_non
# MCN = MCNS
# MEN = MN
# MGA = MGA
# MPG = MPGN
# MSP = MesPro
# PUR = Purpura
# TIN = TIN
# XXX_Blank = blank


In [ ]:
annotated_dict = dict(zip(np.arange(len(annotated_group)), annotated_group))
annotated_dict

In [ ]:
NUM_CLASSES = len(annotated_group)
IMG_SIZE = 380
BATCH_SIZE = 64
WORKERS = 32

In [ ]:
def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB4(include_top=False, input_tensor=inputs, weights=None,
                           drop_connect_rate=0.3)
    model.trainable = False
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.Dropout(0.2, name="top2_dropout")(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2, name="top1_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax', name="pred")(x)
    
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

with strategy.scope():
    model = build_model()
model.summary()

In [ ]:
# load weight
# set path to the trained weight

model.load_weights(#set path to 'CNN_Diag_5fold_weight_1st.h5' - 'CNN_Diag_5fold_weight_5th.h5')

In [ ]:
# set image_directory 
# assumed directory structure is 'YOUR_TEST_DIRECTORY/SUB_DIRECTORY/IMAGES'

test_dir = # YOUR_TEST_DIRECTORY

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    target_size=(IMG_SIZE,IMG_SIZE), 
                                                    batch_size=BATCH_SIZE,
                                                    color_mode="rgb",
                                                    shuffle = False,
                                                    class_mode='categorical',
                                                   interpolation = 'bilinear')

In [ ]:
test_filenames = test_generator.filenames
test_filenames

In [ ]:
predicted_scores = model.predict_generator(test_generator,workers=WORKERS, verbose=1)
predicted_labels = np.argmax(predicted_scores, axis=1)
predicted_probabilities = []
for i, j in zip(predicted_scores, predicted_labels):
    predicted_probabilities.append(i[j])

In [ ]:
predicted_scores

In [ ]:
# See ’annotated_dict' for the meanings of predicted_labels
predicted_labels

In [ ]:
predicted_probabilities

In [ ]:
plt.hist(predicted_probabilities, bins = 50)